<a href="https://colab.research.google.com/github/brando710/brando710.github.io/blob/main/WhatsApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

My New Server

In [ ]:
import socket
import threading

HOST = "192.168.1.197"
PORT = 5555

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen()

clients = []
usernames = []


def broadcast(message):
    """Send message to all clients safely."""
    for client in clients[:]:
        try:
            client.send(message)
        except:
            clients.remove(client)
            client.close()


def handle_client(client):
    while True:
        try:
            message = client.recv(1024)
            decoded = message.decode("utf-8")

            # Private message
            if decoded.startswith("@"):
                parts = decoded.split(" ", 1)
                target_name = parts[0][1:]

                if len(parts) > 1:
                    msg_body = parts[1]
                else:
                    continue

                if target_name in usernames:
                    index = usernames.index(target_name)
                    target_client = clients[index]
                    target_client.send(
                        f"[PRIVATE] {msg_body}".encode("utf-8")
                    )
            else:
                broadcast(message)

        except:
            index = clients.index(client)
            username = usernames[index]

            clients.remove(client)
            usernames.remove(username)
            client.close()

            broadcast(f"{username} left the chat.".encode("utf-8"))
            print(f"[-] {username} disconnected")
            break


def receive_connections():
    print(f"Server running on {HOST}:{PORT}")

    while True:
        client, address = server.accept()
        print(f"Connected with {address}")

        client.send("USERNAME".encode("utf-8"))
        username = client.recv(1024).decode("utf-8")

        usernames.append(username)
        clients.append(client)

        print(f"[+] {username} joined from {address}")
        broadcast(f"{username} joined the chat!".encode("utf-8"))

        thread = threading.Thread(target=handle_client, args=(client,))
        thread.start()


receive_connections()

My New client

In [ ]:
# chat_client.py

# Allows the program to communicate over network connections (TCP/IP). Needed to connect to the chat server
# Allows running multiple tasks simultaneously. Since we want to send and receive at the same time!
import socket
import threading

#	HOST = server IP address.
# 127.0.0.1 means connect to server running on same machine.
# PORT = communication channel number.
# Must match server settings.
HOST = "192.168.1.197"
PORT = 5555

username = input("Choose your username: ")

# Creates a TCP socket.
# AF_INET = IPv4.
# SOCK_STREAM = TCP (reliable connection).
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

#	Establishes connection to chat server.
#	If server is not running → connection fails.
client.connect((HOST, PORT)

#	Defines function responsible for listening to server messages.
def receive_messages():
    while True: # infinite loop, client always listens for the incoming messages
        try:
            message = client.recv(1024).decode("utf-8") # Waits for message from server. Receives up to 1024 bytes. Converts binary data → readable text.

            #handle handshake
            if message == "USERNAME": #	Server sends “USERNAME” when a new client connects. This is a signal asking for the user’s name.
                client.send(username.encode("utf-8")) # Sends username back to server. Encodes text into bytes for network transmission.
            else:
                print(message) # If message is normal chat text → display it.

        except: #handling connection errors
            print("Connection closed.")
            client.close() #close socket
            break

# function: sending message
def write_messages():
    while True:
        msg = input("")
        if msg.lower() == "/quit":
            client.close()
            break
        message = f"{username}: {msg}"
        client.send(message.encode("utf-8"))

threading.Thread(target=receive_messages).start() # Start background thread for receiving messages. Allows messages to appear while user types.
threading.Thread(target=write_messages).start() # Start another thread for sending messages.